<div align="center">

  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>

  [中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [Türkçe](https://docs.ultralytics.com/tr/) | [Tiếng Việt](https://docs.ultralytics.com/vi/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml"><img src="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml/badge.svg" alt="Ultralytics CI"></a>
  <a href="https://colab.research.google.com/github/ultralytics/notebooks/blob/main/notebooks/how-to-count-the-objects-using-ultralytics-yolo.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  
  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>
  
  Welcome to the Object counting using Ultralytics YOLO11 🚀 notebook! <a href="https://github.com/ultralytics/ultralytics">YOLO11</a> is the latest version of the YOLO (You Only Look Once) AI models developed by <a href="https://ultralytics.com">Ultralytics</a>. We hope that the resources in this notebook will help you get the most out of YOLO11. Please browse the YOLO11 <a href="https://docs.ultralytics.com/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!</div>

# Object Counting using Ultralytics YOLO11

This notebook serves as a starting point for [counting objects](https://docs.ultralytics.com/guides/object-counting/) in videos or live streams using the YOLO11 model.

### What is Object Counting?

- Object counting with YOLO11 involves accurate identification and counting of specific objects in videos and camera streams. YOLO11 excels in real-time applications, providing efficient and precise object counting for various scenarios like crowd analysis and surveillance, thanks to its state-of-the-art algorithms and deep learning capabilities.

### Advantages of Object Counting?

- **Resource Optimization**: Object counting facilitates efficient resource management by providing accurate counts, and optimizing resource allocation in applications like inventory management.
- **Enhanced Security**: Object counting enhances security and surveillance by accurately tracking and counting entities, aiding in proactive threat detection.
- **Informed Decision-Making**: Object counting offers valuable insights for decision-making, optimizing processes in retail, traffic management, and various other domains.

### Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [3]:
import ultralytics
import cv2
import matplotlib.pyplot as plt
from ultralytics import solutions

ultralytics.checks()

Ultralytics 8.3.74  Python-3.10.11 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i7-11700 2.50GHz)
Setup complete  (16 CPUs, 31.9 GB RAM, 259.4/476.1 GB disk)
Setup complete  (16 CPUs, 31.9 GB RAM, 259.4/476.1 GB disk)


### Read the Video File

- You can either read the video file directly or stream the content from an RTSP (Real-Time Streaming Protocol) source, allowing for flexible video input depending on your needs.
- We will also set up the video writer to handle the output video writing.

In [44]:
cap = cv2.VideoCapture("../../asset/G1_Grbng_Masuk_Brt_NVR250_LG_20250205070020_20250205070200_1608394.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS)
)

# Video writer
video_writer = cv2.VideoWriter("counting.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

### Find The Region Coordinates

In [25]:
def get_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:  # Saat mouse double-click
        print(f"Mouse Hovering at: ({x}, {y})")

In [26]:
success, imx = cap.read()

# Buat window dan set event callback
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", get_coordinates)

while True:
    cv2.imshow("Image", imx)
    if cv2.waitKey(1) & 0xFF == 27:  # Tekan ESC untuk keluar
        break

cv2.destroyAllWindows()

Mouse Hovering at: (729, 273)
Mouse Hovering at: (1286, 395)
Mouse Hovering at: (712, 229)
Mouse Hovering at: (1383, 378)


In [ ]:
cv2.waitKey(0)

### Define Region Coordinates

Here, we set the coordinates for specific regions to ensure accurate object tracking and analysis within the video or stream. This helps monitor and count objects effectively in different areas.

In [40]:
# Define region points
region_points = [(712, 229), (1383, 378)]  # For line counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

### Initialize the ObjectCounter Class

- Now, let's initialize the `ObjectCounter` class to track and count objects in each frame of the video.

In [43]:
# Init ObjectCounter
counter = solutions.ObjectCounter(
    show=True,  # Display the output
    region=region_points,  # Pass region points
    model="yolo11x.pt",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    classes=[1, 2, 3, 5, 6, 7, 8, 17, ],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    # show_in=True,  # Display in counts
    # show_out=True,  # Display out counts
    line_width=2,  # Adjust the line width for bounding boxes and text display
)

Ultralytics Solutions:  {'region': [(712, 229), (1383, 378)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': True, 'model': 'yolo11x.pt', 'classes': [1, 2, 3, 5, 6, 7, 8, 17], 'line_width': 2}


100%|██████████| 109M/109M [03:50<00:00, 497kB/s]  


### Process Video Frames

In this step, we will process each frame of the video to detect and analyze objects. This allows for real-time tracking and counting, based on the visual data in the frames.

In [45]:
cv2.namedWindow("Ultralytics Solutions", cv2.WINDOW_NORMAL)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    im0 = counter.count(im0)  # count the objects
    video_writer.write(im0)   # write the video frames

cap.release()   # Release the capture
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 10 cars, 363.5ms
Speed: 2.0ms preprocess, 363.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 351.4ms
Speed: 1.0ms preprocess, 351.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 353.8ms
Speed: 2.0ms preprocess, 353.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 380.6ms
Speed: 2.0ms preprocess, 380.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 334.7ms
Speed: 2.0ms preprocess, 334.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 338.0ms
Speed: 2.0ms preprocess, 338.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 motorcycle, 340.6ms
Speed: 2.0ms preprocess, 340.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 motorcycle, 352.3ms
Speed: 2.0ms preprocess, 352.3ms inference, 1.0ms postproces

![Fish Counting in the Sea Using Ultralytics YOLO11](https://github.com/ultralytics/docs/releases/download/0/fish-counting-in-sea-using-ultralytics-yolov8.avif)